In [15]:
# 4.6 serialization(직렬화, 데이터나 객체를 저장 가능한 형태 또는 전송 가능한 형태로 바꾸는 과정)
# chat 설정 저장하는 방법, 불러오는 방법

from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
#     temperature=0.1,
#     max_tokens=450,
#     model="gpt-4o-mini"
# )

# 설정 저장하기
# chat.save("model_gpt4_4o_mini.json")

# 설정 불러오기
chat = load_llm("model_gpt4_4o_mini.json")

chat.predict("Hi! I'm Sumin")

c:\Users\Sumin\Desktop\development\nomad_GPT\FULLSTACK-GPT\envGPT\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Sumin\Desktop\development\nomad_GPT\FULLSTACK-GPT\envGPT\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'Hi Sumin! How can I assist you today?'

In [ ]:

# 4.6 
# OpenAI를 사용할 때 비용 측정 방법
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(False)

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for sausage")
    b = chat.predict("What is the recipe for macaroon")
    print(a, b, "\n")
    print(usage)
    print("total cost:", usage.total_cost)
    print("total token:", usage.total_tokens)
    print("prompt tokens:", usage.prompt_tokens)
    print("completion tokens:", usage.completion_tokens)



In [ ]:

# 4.5 유용한 기능 소개
# set_llm_cache(InMemoryCache()): 답변을 저장함. 같은 질문을 받을 경우, 저장된 답변을 바로 출력함. 시스템을 껐다 켜면 사라짐.
# set_llm_cache(SQLiteCache("원하는이름.db")): 답변 저장. 파일로 저장하기 때문에 껐다 켜도 사용 가능.
# 이 외에도 다양한 cache를 사용할 수 있음. langchain에서는 다양한 third party를 제공하고 있음. 또한, 직접 cache를 만들어서 쓸 수도 있음.
# set_debug(True): llm에서 어떤 일이 일어나고 있는지 출력해줌. 답변이 나온 이유, 어떤 모델을 쓰고 있는지 등등.
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler(), ],
)

chat.predict("How do you make korea's Braised ribs")



In [ ]:

#4.4 메모리 합치기(많은 프롬프트를 하나로 합치기)
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrg! That is a secret!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

# chain.invoke(
{
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrg! That is a secret!",
    "question":"What is your fav food?",
}
)


In [ ]:

#4.4 파일에서 형식 불러오기
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")
prompt.format(country="North Korea")


In [ ]:
# 4.3
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "what do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix= "What do you know about {country}?", #형식화된 예제의 마지막에 나오는 내용
    input_variables=["country"]
)

prompt.format(country="Brazil")


In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "what do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix= "What do you know about {country}?", #형식화된 예제의 마지막에 나오는 내용
    input_variables=["country"]
)

prompt.format(country="Brazil")

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answer."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country":"Thailand",
})



In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)
# chat.predict("What do you know about India?")


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "what do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 1. 예제 형식 지정하기(좀 더 명확하게)
# example_template = """
#     Human: {question}
#     AI: {answer}
# """

# example_prompt = PromptTemplate.from_template(example_template)

# 2. 예제 형식 지정하기(간단)
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples, #example_prompt 와 examplaes를 가져와 형식화(formatting)함. 
    suffix= "What do you know about {country}?", #형식화된 예제의 마지막에 나오는 내용
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country":"Turkey",
})

In [ ]:
from langchain.prompts import PromptTemplate

# PromptTemplate Using #1 ======================
# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="India")

# PromptTemplate Using #2 ======================
# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables=["country"],
# )
# t.format(country="India")


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

poet_template = ChatPromptTemplate.from_messages([
    ("system", "You are a poet. You don’t write anything other than poetry."),
    ("human", "Write me a poem. The theme of the poem is {language}.")
])

poet_chain = poet_template | chat

explain_template = ChatPromptTemplate.from_messages([
    ("system", "You are a critic. Read the poem and evaluate what the poem is like."),
    ("human", "{poem}")
])

explain_chain = explain_template | chat

final_chain = {"poem": poet_chain} | explain_chain

final_chain.invoke({
    "language":"javascript"
})